In [1]:
import os

In [2]:
%pwd

'/Users/vivek/work/GIT/Triluxo/reserch'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vivek/work/GIT/Triluxo'

In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
index = "triluxo"


In [8]:
api_key = os.getenv("PINECONE")

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec


index_name = "hybrid-search-pinecone"

#initizing the pinecone clinet
pc = Pinecone(api_key=api_key)

try:
    # Retrieve the list of existing indexes
    existing_indexes = pc.list_indexes()
    print(f"Existing indexes: {existing_indexes}")

    if index_name not in existing_indexes:
        # Create the index if it doesn't exist
        pc.create_index(
            name=index_name,
            dimension=384,  # Dimension of your dense model
            metric='dotproduct',  # Metric for similarity search
            spec=ServerlessSpec(cloud='aws', region='us-east-1')
        )
        print(f"Index '{index_name}' created successfully.")
    else:
        print(f"Index '{index_name}' already exists.")

except Exception as e:
    if e.status == 409:
        # Handle the case where the index already exists
        print(f"Index '{index_name}' already exists. Proceeding to use it.")
    else:
        # Handle other potential exceptions
        print(f"An error occurred while creating the index: {e}")

/opt/miniconda3/envs/triluxo/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Existing indexes: [{
    "name": "hybrid-search-pinecone",
    "dimension": 384,
    "metric": "dotproduct",
    "host": "hybrid-search-pinecone-nqxn8hs.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}]


In [12]:
index = pc.Index(index_name)
index

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")


In [16]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [17]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default() #defining the default encoder i.e .default()
bm25_encoder

KeyboardInterrupt: 

In [ ]:
# Function to convert text file to Python list
def file_to_list(filename):
    with open("output.txt", 'r') as file:
        # Read the file content
        content = file.read()
        # Split the content by lines and return as a list
        lines_list = content.splitlines()  # Each line will be a separate item in the list
    return lines_list

# Example usage
filename = 'example.txt'  # Replace with your text file name
result_list = file_to_list(filename)
print(result_list)